<a href="https://colab.research.google.com/github/m0tchy/camera-geometry-tutorial/blob/main/08_camera_pose(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

今回は、世界座標系として +Z を上向きにとってプロットすることにする。そのため、 YZ を入れ替えるような特殊な処理は必要なくなる。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def init_3d_plot():
    """Create a new figure for a 3d plot."""

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection="3d")
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_zlim(-1, 9)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.view_init(20, -25)

    # X, Y, Z axes
    ax.plot([0, 1], [0, 0], [0, 0], color="RED")  # X 軸
    ax.plot([0, 0], [0, 1], [0, 0], color="GREEN")  # Y 軸
    ax.plot([0, 0], [0, 0], [0, 1], color="BLUE")  # Z 軸

    return fig, ax


# demo
fig, ax = init_3d_plot()

家のモデルも定義しよう。
前回のとは違い、 Z 軸が高さになるように描かれている。

In [ ]:
# (0, 0, 0) が床面の中央なっている家のモデル
house_vertices = np.array([
    [-1, -1, -1, 1], 
    [1, -1, -1, 1], 
    [1, 1, -1, 1], 
    [-1, 1, -1, 1], 

    [-1, -1, 1, 1],
    [1, -1, 1, 1],
    [1, 1, 1, 1],
    [-1, 1, 1, 1],

    [0, 0, 2, 1],
])

house_edges = [
    [0, 1, 2, 3, 0], 
    [4, 5, 6, 7, 4], 
    [0, 4, 8], 
    [1, 5, 8], 
    [2, 6, 8], 
    [3, 7, 8], 
]


このようなモデルを変換してプロットする関数を作る。

In [ ]:
def plot_model(ax, vertices_, edges, transform=np.identity(4)):
    X_ = (transform @ vertices_.T).T

    # 普通の座標に変換
    Xw = X_[:, 0] / X_[:, 3]
    Yw = X_[:, 1] / X_[:, 3]
    Zw = X_[:, 2] / X_[:, 3]

    ax.scatter(Xw, Yw, Zw, color="GRAY")
    for e in edges:
        ax.plot(Xw[e], Yw[e], Zw[e], color="GRAY")

# モデルの位置を変える
house_trans1 = np.identity(4)  # 単位行列なので、位置を変更しない

house_trans2 = np.array([
    [1, 0, 0, -2],
    [0, 1, 0, 0],
    [0, 0, 1, 4],
    [0, 0, 0, 1],
])

# demo
fig, ax = init_3d_plot()
plot_model(ax, house_vertices, house_edges, house_trans1)
plot_model(ax, house_vertices, house_edges, house_trans2)

# カメラの姿勢



## 姿勢を表す行列

カメラの姿勢は、世界座標からカメラ座標の変換行列で定義される。
空間の点の世界座標とカメラ座標をそれぞれ $\boldsymbol{X}^\mathsf{W}$, $\boldsymbol{X}^\mathsf{C}$ とすると、一般に
$$
\boldsymbol{X}^\mathsf{C} = R \boldsymbol{X}^\mathsf{W} + \boldsymbol{T}
$$
の形になる。
この $R$, $\boldsymbol{T}$ のことをカメラの**姿勢** (camera pose) という。

$R$ が任意の行列（つまり線形変換）になることもあるが、普通は回転行列である。その場合、この式は剛体変換（鏡映を含まないユークリッド変換）である。
ここでは $R$ が回転行列の場合だけを考えることにする。

また、同次座標では以下のようになる。
\begin{align}
    \tilde{\boldsymbol{X}}^\mathsf{C} &=
    \begin{bmatrix}
        R & \boldsymbol{T} \\
        \boldsymbol{0}^\top & 1
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^\mathsf{W}
\end{align}




## 撮影されたシーンの表示

カメラを定義できれば、透視投影を計算してシーンを撮影することができる。

投影面の座標系の設定と頂点の変換・辺の表示は前回と同じだが、
それぞれ関数にして実装しよう。

In [ ]:
def init_2d_plot():
    fig, ax = plt.subplots()
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.invert_yaxis()  # 画像の座標系になるように y 軸を下向きにする
    ax.set_aspect("equal")

    return fig, ax


def plot_image(ax, vertices_, edges, transform, camera_pose, focal_length=1):

    # 透視投影行列
    projection = np.array([
    [focal_length, 0, 0, 0], 
    [0, focal_length, 0, 0], 
    [0, 0, focal_length, 0], 
    [0, 0, 1, 0]])

    # X はモデル変換・ビュー変換・透視投影の順で変換される
    X_ = (projection @ camera_pose @ transform @ vertices_.T).T

    # 普通の座標に変換
    x = X_[:, 0] / X_[:, 3]
    y = X_[:, 1] / X_[:, 3]

    ax.scatter(x, y, color="BLACK")
    for e in edges:
        ax.plot(x[e], y[e])  # 表示のために Y, Z を入れ替える


## 何も変換しない場合

前回は世界座標と一致したカメラ座標を回転・並進するとどうなるかを考えた。
もし、上の式で $R = I$、 $\boldsymbol{T}=\mathbf{0}$ とすると、これは何も変換しないことになる。
$$
\boldsymbol{X}^\mathsf{C} = \boldsymbol{X}^\mathsf{W}
$$

\begin{align}
    \tilde{\boldsymbol{X}}^\mathsf{C} &=
    \begin{bmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^\mathsf{W}
\end{align}


In [ ]:
pose = np.identity(4)  #  R=I, T=0

fig, ax = init_2d_plot()

# 原点にある家は、カメラと重なるのでうまく表示されない
#plot_image(ax, house_vertices, house_edges, house_trans1, pose)
plot_image(ax, house_vertices, house_edges, house_trans2, pose)

`house_trans1` は何も移動しないので、カメラと物体が重なった位置になる。この場合（ちゃんと処理を書かないと）うまく表示されない。

カメラは Z 軸方向を向いているので、`house_trans2` は下から見上げたように表示されることがわかる。

## カメラの向きは変わらず位置だけ変わる場合

カメラの位置は、その光学中心（投影中心）で表す。
カメラ位置が世界座標の原点にあるとし、世界座標 $\boldsymbol{C}$ にするには、
撮影対象の世界座標をその**逆**に動かすことでカメラから見た座標に変換できる。したがって、
$$
\boldsymbol{X}^\mathsf{C} = \boldsymbol{X}^\mathsf{W} - \boldsymbol{C}
$$
\begin{align}
    \tilde{\boldsymbol{X}}^\mathsf{C} &=
    \begin{bmatrix}
        I_{3\times 3} & -\boldsymbol{C} \\
        \boldsymbol{0}_3^\top & 1
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^\mathsf{W}
\end{align}
となる。

例えば、カメラの位置を Z 軸の負の方向に動かすと、先ほどの図形は小さく映る。
物体がカメラの前方に配置されるようにすれば、 `house_trans1`, `house_trans2` はそれぞれ正しく表示される。

In [ ]:
C = np.array([0, 0, -3])  # カメラの位置

pose = np.identity(4)  #  R=I, T=0
pose[0:3, 3] = -C  # T の部分を変更

fig, ax = init_2d_plot()

# 物体は、カメラと重ならなければ正しく映る
plot_image(ax, house_vertices, house_edges, house_trans1, pose)
plot_image(ax, house_vertices, house_edges, house_trans2, pose)

##  位置は動かず向きだけ変わる場合

カメラの向きが変わると、座標軸のさす方向が変わる。
例えば、カメラ座標系では X 軸はいつも方向ベクトル $[1, 0, 0]$ で表現できる。
このベクトルが世界座標系では違う向きになっている。

軸の方向ベクトルが世界座標系とカメラ座標系で一致している状態から回転させて向きを変えることを考えよう。
それは回転行列で表現できる。
例えば、カメラを右に向けると、撮影対象のカメラ座標は世界座標から左に回転した位置になる。

カメラの回転を回転行列 $V$ とすると、その**逆行列**を考えればよい。ここで回転行列の性質から $V^{-1} = V^\top$ なので、
$$
\boldsymbol{X}^\mathsf{C} = V^\top \boldsymbol{X}^\mathsf{W}$$
となる。



In [ ]:
ang=np.radians(-10)  # Y 軸周りの回転
V = np.array([
    [np.cos(ang), 0, np.sin(ang)],
    [0, 1, 0],
    [-np.sin(ang), 0, np.cos(ang)],
])
pose = np.identity(4)  #  R=I, T=0
pose[0:3, 0:3] = V.T  # R の部分を変更

fig, ax = init_2d_plot()

# 物体は、カメラと重ならなければ正しく映る
# plot_image(ax, house_vertices, house_edges, house_trans1, pose)
plot_image(ax, house_vertices, house_edges, house_trans2, pose)

### 練習

右に $\theta$ 度, 下に $\phi$ 度首を振る回転（パンチル）は世界座標系の $\mathsf{Y}^\mathsf{W}$ 軸回転行列 $R_{\mathsf{Y}}(\phi)$, 世界座標系の $\mathsf{X}^\mathsf{W}$ 軸回転行列 $R_{\mathsf{X}}(\theta)$ を使うと、$V = R_{\mathsf{Y}}(\phi) R_{\mathsf{X}}(\theta)$ である。したがって、 $V^\top = R_\mathsf{X}(\theta)^\top R_\mathsf{Y}(\phi)^\top  $ となる。

パンチル回転を表す回転行列を作る関数  `rotation_pan_tilt(pan, tilt)` を定義して、テストしなさい。

## 一般の場合

この2つが組み合わされた時は、 $\boldsymbol{X}^\mathsf{C} = V^\top\boldsymbol{X}^\mathsf{W} - \boldsymbol{C}$ **ではない** ことに注意しよう。
この変換では先に回転が行われる。そうすると、すでに座標系が動いているため、平行移動したい先の $\boldsymbol{C}$ も移動している。その移動先は、 $V^\top \boldsymbol{C}$ である。したがって、
\begin{align}
    \boldsymbol{X}^\mathsf{C} &= V^\top\boldsymbol{X}^\mathsf{W} - V^\top \boldsymbol{C} \\
    &= V^\top(\boldsymbol{X}^\mathsf{W} - \boldsymbol{C})
\end{align}
である。
2番目の式は先に平行移動してカメラ中心を原点に移動してから回転すると解釈できる。

最初の一般形との関係は、
\begin{align}
    R &= V^\top \\
    \boldsymbol{T} &= -V^\top \boldsymbol{C}
\end{align}
となる。
また、同次座標では以下のようになる。
\begin{align}
    \tilde{\boldsymbol{X}}^\mathsf{C} &=
    \begin{bmatrix}
        R & \boldsymbol{T} \\
        \boldsymbol{0}^\top & 1
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^\mathsf{W} \\
    &= \begin{bmatrix}
        R & -R\boldsymbol{C} \\
        \boldsymbol{0}^\top & 1
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^\mathsf{W}
\end{align}
この2番目の式のように、回転行列を転置せず、投影中心を含めた形で書くことも多い。

**補足**: 前回の資料は $V$ に相当するものを $R$ と書いていた。混乱しないように。

まずは、適当にカメラを設置する。
ここでは、カメラの向きを以下のようにする。
* 右方向 ($\mathsf{X}^\mathsf{C}$) は世界座標系と同じにする
* $\mathsf{Z}^\mathsf{C}$ 軸方向を世界座標系の $\mathsf{Y}^\mathsf{C}$ にする。

これはつまり、世界座標を $\mathsf{X}^\mathsf{W}$ 軸回りに $-90^\circ$ 回転することに相当する。

In [ ]:
# カメラを定義するために、姿勢を表す行列（＝世界 to カメラ変換）を作る
center = np.array([-2, -4, 3.5])  # in world
# X 軸周り -90 度の回転を、転置して逆変換にする
V = np.array([
    [1, 0, 0],
    [0, 0, 1],
    [0, -1, 0],
])

R = V.T
T = -V.T @ center

pose = np.identity(4)
pose[0:3, 0:3] = R
pose[0:3, 3] = T

print(pose)

fig, ax = init_2d_plot()
plot_image(ax, house_vertices, house_edges, house_trans1, pose)
plot_image(ax, house_vertices, house_edges, house_trans2, pose)

## 姿勢行列からカメラの姿勢を調べる

次に3次元空間上のカメラの姿勢を表示したい。
カメラの姿勢は以下の行列の形で与えられているとする。

$$
    \begin{bmatrix}
        R & \boldsymbol{T} \\
        \boldsymbol{0}^\top & 1
    \end{bmatrix}
$$

そこから世界座標系の中でのカメラ中心や
座標軸の向きを調べる必要がある。


### カメラの位置

カメラの位置、つまり光学中心は上の関係式を変形すると $R$, $\boldsymbol{T}$ から
\begin{align}
    \boldsymbol{C} = -R^\top \boldsymbol{T}
\end{align}
で計算できる。

### カメラ座標系の座標軸の方向

上の座標変換を変形して、カメラ座標から世界座標への逆変換を考えよう。

\begin{align}
    \boldsymbol{X}^{\mathsf{C}} &= 
    R \boldsymbol{X}^{\mathsf{W}} + \boldsymbol{T}  \\ 
    \Longleftrightarrow
    \boldsymbol{X}^{\mathsf{W}} &=  R^\top (\boldsymbol{X}^{\mathsf{C}} - \boldsymbol{T}) \\
    \Longleftrightarrow
    \boldsymbol{X}^{\mathsf{W}} &=  R^\top \boldsymbol{X}^{\mathsf{C}} - R^\top\boldsymbol{T} \\
    \Longleftrightarrow
    \boldsymbol{X}^\mathsf{W} &= R^\top\boldsymbol{X}^\mathsf{C} + \boldsymbol{C}
\end{align}

この式で、カメラ座標で表される点の位置を世界座標に変換できる。

$\mathsf{X}^\mathsf{C}$ 軸の方向はカメラ座標の $[1, 0, 0]$ を世界座標に変換すればよい。
すると、$R^\top [1, 0, 0]^\top - \boldsymbol{C}$ となる。
これは、$\mathsf{X}^\mathsf{C}$ 軸が $\boldsymbol{C}$ を基準に $R^\top [1, 0, 0]^\top$ の方向であることを表している。この式は $R^\top$ の第1列、つまり $R$ の第1行を取り出す。

ここで、
$$
    R = \begin{bmatrix}
        \boldsymbol{R}_1^\top \\
        \boldsymbol{R}_2^\top \\
        \boldsymbol{R}_3^\top \end{bmatrix}
$$
のように**行ベクトル**に分けると、 $R^\top [1, 0, 0]^\top = \boldsymbol{R}_1^\top$ である。
したがって、 $R$ の各**行**はカメラの $\mathsf{X}^\mathsf{C}$, $\mathsf{Y}^\mathsf{C}$, $\mathsf{Z}^\mathsf{C}$ 軸それぞれの**世界座標系での**方向ベクトルになっていることがわかる。


## カメラの表示

与えられた姿勢行列から、カメラの座標系をプロットする関数を作ろう。

In [ ]:
def plot_camera(ax, pose, focal_length=1.0):

    assert pose.shape == (4, 4)
    assert (pose[3] == np.array([0, 0, 0, 1])).all()

    # aliases
    R = pose[0:3, 0:3] / pose[3, 3]
    T = pose[0:3, 3] / pose[3, 3]
    assert T.shape == (3,)

    # camera center in world
    C = -R.T @ T

    # camera axes in world
    X = C + R[0]
    Y = C + R[1]
    Z = C + R[2]

    # camera to world coordinate transform
    pose_inv = np.identity(4)
    pose_inv[0:3, 0:3] = R.T
    pose_inv[0:3, 3] = C

    # 数値的には、以下のように逆行列を求める関数を使うこともできるが、
    # 剛体変換であれば、上のように行列を作るほうが良い
    # pose_inv = np.linalg.inv(pose)

    # draw three axes
    #ax.plot([0, 0], [-1, 9], [0, 0], ":", color="SILVER")
    ax.plot([C[0], X[0]], [C[1], X[1]], [C[2], X[2]], color="RED")    # X 軸
    ax.plot([C[0], Y[0]], [C[1], Y[1]], [C[2], Y[2]], color="GREEN")  # Y 軸
    ax.plot([C[0], Z[0]], [C[1], Z[1]], [C[2], Z[2]], color="BLUE")   # Z 軸

    # カメラ座標系での投影面の定義
    screen_vertices_c = np.array([
        [-1, -1, focal_length, 1],
        [1, -1, focal_length, 1],
        [1, 1, focal_length, 1],
        [-1, 1, focal_length, 1]])
    # カメラ座標から世界座標へ変換する
    screen_vertices = (pose_inv @ screen_vertices_c.T).T

    screen_edges = [0, 1, 2, 3, 0]

    ax.plot(screen_vertices[screen_edges, 0], 
            screen_vertices[screen_edges, 1], 
            screen_vertices[screen_edges, 2], color="GRAY")


In [ ]:
# 世界座標とカメラを表示する
fig, ax = init_3d_plot()
plot_camera(ax, pose)

# 物体もついでに
plot_model(ax, house_vertices, house_edges, house_trans1)
plot_model(ax, house_vertices, house_edges, house_trans2)

## カメラの姿勢を軸の方向で指定

カメラの姿勢を、各軸の方向で直接定義できると便利である。
各軸の方向を行として並べたものを $R$ とし、
カメラの位置 $\boldsymbol{C}$ から行列を計算する。
ただし、各軸が直交していないとおかしいカメラになってしまうので注意が必要である。

カメラの姿勢行列を作る関数を定義しよう。
ここでは、 $\mathsf{X}^\mathsf{C}$, $\mathsf{Y}^\mathsf{C}$, $\mathsf{Z}^\mathsf{C}$ の方向ベクトルを left, down, front vector と呼ぶことにする。

In [ ]:
def compose_camera_pose(center, left, down, front):
    assert left.shape == (3,)
    assert down.shape == (3,)
    assert front.shape == (3,)

    R = np.array([left, down, front])
    T = -R @ center
    assert T.shape == (3,)

    pose = np.identity(4)
    pose[0:3, 0:3] = R
    pose[0:3, 3] = T

    return pose


In [ ]:
# カメラを定義するために、姿勢を表す行列（＝世界 to カメラ変換）を作る
# 座標はすべて世界座標で指定する
# 直交するようにしないといけない
pos = np.array([-2, -4, 3.5])
left = np.array([1, 0, 0])
down = np.array([0, 0, -1])
front = np.array([0, 1, 0])
camera_pose = compose_camera_pose(pos, left, down, front)

# 世界座標とカメラを表示する
fig, ax = init_3d_plot()
plot_camera(ax, camera_pose)

## 発展課題：注視点による指定

カメラを特定の位置に向かせたいということは多い。
そこで、カメラの位置と注視点を指定することでカメラの姿勢を定義する。
ただし、この2つの情報では姿勢は一意に定まらない。
同じ方向でも方向軸周りの回転の自由度が生まれる（この回転は[roll](https://ja.wikipedia.org/wiki/%E3%83%AD%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0) と呼ばれることがある）。
しかし、重力のある空間の中では、通常カメラの上下方向は地面に垂直（つまり鉛直）にすることが多いだろう。
そこで、CG の分野では、
地面の方向（もしくは空の方向）も指定することで、カメラの姿勢を定義する。
例えば、 OpenGL では [gluLookAt 関数](https://www.khronos.org/registry/OpenGL-Refpages/gl2.1/xhtml/gluLookAt.xml) が用意されている。

実装は以下の順番で考える。
0. $R = I$, $\boldsymbol{T} = \mathbf{0}$ から始める（世界座標系と一致）
1. $\mathsf{Z}^\mathsf{C}$ 軸 (`front`) を視線方向に向ける。これはパンチル回転で行えばよい。
2. その座標系で $\mathsf{Z}^\mathsf{C}$ 軸周りの回転を行い、 $\mathsf{Y}^\mathsf{C}$ を `gravity` に向ける。具体的には、$\mathsf{Y}$-$\mathsf{Z}^\mathsf{C}$ 平面に `gravity` が含まれるようにする。 $\mathsf{Y}^\mathsf{C}$ 軸が `gravity` と平行になるわけではないことに注意。

3つの軸が常に直交になるようにするには、 R を単位行列 I から初めて、何度か回転行列を掛けていくことでできる。